# Train-Test Split and Cross Validation - using numpy
## Maaz Ali Nadeem - i200452 - BS (AI-5J) - Machine Learning Lab Task 1

In [1]:
import numpy as np
import pandas as pd
import random

In [88]:
#Set a ratio for the size of train and test dataframes
test_ratio = 0.3
train_ratio = 0.7

In [2]:
ecg_df = pd.read_csv("ECG200_TRAIN.csv", delimiter="  ", header=None)
ecg_df[0].value_counts()

D:\Python_PAI\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


 1.0    69
-1.0    31
Name: 0, dtype: int64

In [35]:
df_label1 = ecg_df[ecg_df[0]==1]
df_label2 = ecg_df[ecg_df[0]==-1]

## K-Folds Verification (k=3)

In [42]:
k = 3
df1 = []
df2 = []

for i in range(k):
    df1.append(pd.DataFrame(np.array_split(df_label1, k)[i]))
    df2.append(pd.DataFrame(np.array_split(df_label2, k)[i]))

In [59]:
for i in range(k):
    
    #Let the kth splitted dataframe be test dataset (all other dataset segments are training)
    test_df = df1[i].append(df2[i], ignore_index = True)
    
    #all other sets are training
    train_df = pd.DataFrame(columns = ecg_df.columns)
    for j in range(k):
        if (j == i):
            continue
        train_df = train_df.append(df1[j], ignore_index=True)
        train_df = train_df.append(df2[j], ignore_index=True)
        
    X_train = train_df.drop([0], axis=1)
    y_train = train_df[0]
    X_test = test_df.drop([0], axis=1)
    y_test = test_df[0]
    
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(X_train, y_train)
    pred=clf.predict(X_test)
    print ("Fold #", i, "Accuracy : " , accuracy_score(y_test,pred)*100) 
    print ("Fold #", i, "F1 Score : ",f1_score(y_test, pred, average='macro')*100)
    print (" ")

Fold # 0 Accuracy :  88.23529411764706
Fold # 0 F1 Score :  85.83333333333333
 
Fold # 1 Accuracy :  72.72727272727273
Fold # 1 F1 Score :  68.57142857142857
 
Fold # 2 Accuracy :  78.78787878787878
Fold # 2 F1 Score :  72.26890756302521
 


## Evaluation with Sklearn

In [60]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import tree

In [61]:
X = ecg_df.drop(0, axis=1)
y = ecg_df[0]
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.3)

In [62]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print ("Accuracy : " , accuracy_score(y_test,pred)*100)  
print("F1 Score : ",f1_score(y_test, pred, average='macro')*100)

Accuracy :  83.33333333333334
F1 Score :  80.74454428754814


## Comparison

The accuracy isnt a good measure of evaluation when dealing with imbalanced data, in this case, the model could 
easily evaluate upto 70% accuracy just by predicting the class label '1' all the time (which is a poor approach)
Therefore i used the F-1 measure which combines both precision and recall scores to evaluate model performance. 

The F-1 scores of the K-folds approach average upto 75.33 while the Sklearn approach yields 80.7. These values fluctuate
more often for Sklearn as the dataset is picked randomly and may have a difference in label proportion each time. 

In [40]:
# def train_test_dataset(df, train_ratio):
    
#     #DataFrame size to generate
#     train_size = int(len(df) * train_ratio)
#     test_ratio = int(len(df) * 1 - train_ratio)
    
#     #Sort the DataFrame according to different labels
#     df = df.sort_values(by=[0])
    
#     df_train = pd.DataFrame(columns=[df.columns])
#     df_test = pd.DataFrame(columns=[df.columns])
    
#     visited_num = []
    
#     while (not len(df_train) == train_size):
#         val = random.randint(0, len(df)-1)
#         if (val in visited_num):
#             continue
#         visited_num.append(val)
#         df_train.append(df.loc[val])
        
#     if (df_train[0].value_counts.tolist()[0]/df_train[0].value_counts.tolist()[1] - df[0].value_counts.tolist()[0]/df[0].value_counts.tolist()[1] <= 8):
#         for i in range(0, len(df)):
#             if i in visited_num:
#                 continue
#             else:
#                 df_test.append(df.loc[i])
                
#         return df_train, df_test
#     else:
#         train_test_dataset(df, train_ratio)